<a href="https://colab.research.google.com/github/ElenaCasolari/hello-streamlit/blob/main/BalanceFinder_PoC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Step 1: geolocalization**
*Libraries: geopy, pandas*

The user insert address, maximum travel distance and keyword. The function search_locations find all the places connected with that keyword inside the bounding box with edge equal to the maximum distance specified. The locations are stored in a data frame and exported to csv.

**Next steps:**
- importing link to web page and additional data for the locations
- implement searching algorithm to search for places with related/similar keywords
- implement queries with multiple keywords

In [100]:
%pip install geopy

import geopy as gp
import pandas as pd

# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Selecting the Geocoding Service: OpenStreetMap
from geopy.geocoders import Nominatim

# Importing the geodesic module from the library
from geopy import distance, point

# Geolocator configuration: selecting the user agent
g = Nominatim(user_agent="BalanceFinder")

#Defining function for searching locations info with keyword inside the  defined area
def search_locations(keyword, area):
    locations = []
    location = g.geocode({keyword}, viewbox=area, bounded=True, extratags=True, exactly_one=False)

    if location:
        for place in location:
            locations.append({
                "Name": place.raw["display_name"],
                "Address": place.address,
                "Latitude": place.latitude,
                "Longitude": place.longitude,
                "Distance [km]": round(distance.great_circle(user_coord, (place.latitude, place.longitude)).km, 2)
            })
    else:
        locations.append({
                "Name": "Non trovato",
                "Address": "Non trovato",
                "Latitude": None,
                "Longitude": None,
                "Distance": None
        })
        print("Non trovato")

    # Creation of DataFrame and exporting to CSV
    df_locations = pd.DataFrame(locations)
    output_csv = df_locations.to_csv(output_csv_file, index=False)
    print("CSV salvato con successo!")

    return df_locations

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [101]:
# Asking the user address and max distance accepted
user_address = g.geocode(input("Address:"))
user_coord = (user_address.latitude, user_address.longitude)

# Calculating the area of interest from user address and max distance
dmax = float(input("Max Transport Distance in km:"))
radmax = dmax / distance.EARTH_RADIUS
print("Max distance in radians", round(radmax, 4), "considering earth radius", round(distance.EARTH_RADIUS), "rad")
pt1 = distance.distance(dmax).destination(user_coord, bearing=225)
pt2 = distance.distance(dmax).destination(user_coord, bearing=45)
area = [point.Point(pt1), point.Point(pt2)]

# Main
keyword = input("Search Keywords:")
path_csv = input("Specify the output file path:")
name_csv = input("Specify the output file name:")
output_csv_file = "/".join([path_csv, name_csv])
locations = search_locations(keyword,area)
locations

Address:Via Buonarroti 1, Buccinasco
Max Transport Distance in km:75
Max distance in radians 0.0118 considering earth radius 6371 rad
Search Keywords:ricicleria
Specify the output file path:/content/drive/MyDrive/Python Driving Licence/BalanceFinder
Specify the output file name:riciclerie
CSV salvato con successo!


,Name,Address,Latitude,Longitude,Distance [km]
0,"Ricicleria, Via dell'Artigianato, Villa Magri,...","Ricicleria, Via dell'Artigianato, Villa Magri,...",45.521119,9.384226,22.96
1,"Ricicleria AMSA, Via Riccardo Lombardi, Parco ...","Ricicleria AMSA, Via Riccardo Lombardi, Parco ...",45.460706,9.065765,5.81
2,"Ricicleria Milizie, Ponte Guido Crepax, Lorent...","Ricicleria Milizie, Ponte Guido Crepax, Lorent...",45.449154,9.156119,3.63
3,"Ricicleria Pedroni, Via Lisiade Pedroni, Affor...","Ricicleria Pedroni, Via Lisiade Pedroni, Affor...",45.511854,9.166968,10.08
4,"Ricicleria Corelli, Via Arcangelo Corelli, Par...","Ricicleria Corelli, Via Arcangelo Corelli, Par...",45.471378,9.253722,11.39


# **Step 2: importing and visualizing routes**
*Libraries: openroutservice, folium, colour, pandas*

A function is defined to create a map that displays in red the user location, while in green the found locations. The routes by car are displayed with different colours.

**Next steps:**
- Displaying on the map the numbers of the found locations
- Adding descriptions to the locations

In [102]:
%%capture
%pip install openrouteservice colour

import openrouteservice as ors
import folium
from colour import Color
import pandas as pd

# Access to ORS
client = ors.Client(key="5b3ce3597851110001cf6248ca796cd70539410cbce5df48f11429b0")

#Creation of the map with user location and keyword locations
def create_map(data):
    map = folium.Map(location=user_coord, zoom_start=8, tiles = "cartodbpositron") #Centered on user
    for _, row in data.iterrows():
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=f"{row['Name']} - {row['Address']}",
            icon=folium.Icon(color='green', icon="refresh")
        ).add_to(map)
    folium.Marker(
        location=user_coord,
        popup="La tua posizione",
        icon=folium.Icon(color="red", icon="user")
    ).add_to(map)

    # Set the viewbox using fit_bounds
    lat_max = max(locations['Latitude']) + 0.1
    lat_min = min(locations['Latitude']) - 0.1
    lon_max = max(locations['Longitude']) + 0.1
    lon_min = min(locations['Longitude']) - 0.1
    bounds = [(lat_min, lon_min), (lat_max, lon_max)]
    map.fit_bounds(bounds)

    return map

In [103]:
# Getting routes from user address to locations
from openrouteservice.directions import directions

# Initialization
sel_place = []
sel_coords = []
sel_route = []

for i in range(len(locations)):
  closest = (locations.iloc[i,2], locations.iloc[i,3])
  coords = ((user_coord[1], user_coord[0]), (closest[1], closest[0])) #directions take locations in the form (lon, lat)
  print(coords)

  route = None
  try:
    route = directions(client, coordinates=coords, profile='driving-hgv', format='geojson')
  except ors.exceptions.ApiError as e:
    print(f"Error calculating route for location {i}: {e}")
    continue  # Skip to the next location

  finally: # Create DataFrames with locations, coordinates and routes
    sel_place.append(closest)
    sel_coords.append(coords)
    sel_route.append(route)

((9.122360622749413, 45.4267551204797), (9.38422632582076, 45.52111855))
((9.122360622749413, 45.4267551204797), (9.06576460686471, 45.460705899999994))
((9.122360622749413, 45.4267551204797), (9.156118741304196, 45.4491544))
((9.122360622749413, 45.4267551204797), (9.166968028318585, 45.5118545))
((9.122360622749413, 45.4267551204797), (9.253722033300988, 45.471377700000005))


In [117]:
# Create map
map = create_map(locations)

# Add routes
for i, route in enumerate(sel_route):
  if "features" in route and len(route["features"]) > 0:
    plot_coords = [list(reversed(coord)) for coord in route["features"][0]['geometry']['coordinates']]
    colours = [color.get_hex() for color in Color("#000000").range_to("#cccccc", len(sel_route))]
    folium.PolyLine(locations = plot_coords, color=colours[i]).add_to(map)
  else:
    print("No route found or unexpected API response.")
    print(route)

# **Step 3: route data analysis**
*Libraries: pandas, folium, openrouteservice*

The travel distance and time is calculated for each destination, together with emissions and cost of transport and added to the map

**Next steps:**
- asking user for preference of type of transport
- adding other impact categories to the study
- importing updated cost of gasoline or other fuels from https://www.cargopedia.net/europe-fuel-prices

In [124]:
# Function for defining travel distance, time and emissions
def route_info():

  destinations = [(user_coord[1], user_coord[0])]
  for i in range(len(locations)):
    destination = (locations.iloc[i,3], locations.iloc[i,2])
    destinations.append(destination)

  info = ors.distance_matrix.distance_matrix(client, destinations, profile='driving-hgv',
                                    sources=[0], destinations=list(range(1,len(destinations))),
                                    metrics=['distance', 'duration'], units='km')
# Function for defining travel distance and time
def route_info():

  destinations = [(user_coord[1], user_coord[0])]
  for i in range(len(locations)):
    destination = (locations.iloc[i,3], locations.iloc[i,2])
    destinations.append(destination)

  info = ors.distance_matrix.distance_matrix(client, destinations, profile='driving-hgv',
                                    sources=[0], destinations=list(range(1,len(destinations))),
                                    metrics=['distance', 'duration'], units='km')

  locations['Travel distance [km]'] = info['distances'][0]
  locations['Travel duration [min]'] = [round(d / 60) for d in info['durations'][0]]

  # Calculating CO2 emissions
  mass = float(input('Tons of material to be transported:'))
  CO2_avgEU = 56.5 # in g/tkm from https://theicct.org/publication/co2-emissions-from-trucks-in-the-eu-an-analysis-of-the-heavy-duty-co2-standards-baseline-data/
  kgCO2 = round(mass*CO2_avgEU/1000*locations['Travel distance [km]'], 2)
  locations['Emissions [kgCO2eq]'] = kgCO2

  # Adding gasoline cost to DataFrame
  cost = 1.829 #taken from https://www.cargopedia.net/europe-fuel-prices
  locations['Cost [€]'] = round(locations['Travel distance [km]']*cost,2)

  # Exporting csv
  output_csv = locations.to_csv(output_csv_file, index=False)
  print("CSV salvato con successo!")

  return locations

# Main
route_info()

# Adding info to map
for index, row in locations.iterrows():
    popup_text = f"<h1>{index + 1}</h1><br>"  # Destination number (starting from 1)
    popup_text += f"<b>Name</b>: {row['Name']}<br>"
    popup_text += f"<b>Address</b>: {row['Address']}<br>"
    popup_text += f"<b>Travel distance</b>: {row['Travel distance [km]']} km<br>"
    popup_text += f"<b>Travel time</b>: {row['Travel duration [min]']} min<br>"
    popup_text += f"<b>Emissions</b>: {row['Emissions [kgCO2eq]']} kgCO2eq<br>"
    popup_text += f"<b>Cost</b>: {row['Cost [€]']} €"

for index, row in locations.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=folium.Popup(popup_text, max_width=300),  # Set popup text
        icon=folium.Icon(color='green', icon="refresh")
    ).add_to(map)

# Output map
map

Tons of material to be transported:10
CSV salvato con successo!
